In [ ]:
import logging

import ssl4rs.utils

logger = ssl4rs.utils.logging.setup_logging_for_analysis_script(logging.INFO)
data_root_path = ssl4rs.utils.config.get_data_root_dir()
data_path = data_root_path / "fmow" / "fmow-rgb" / ".deeplake"
logger.info(f"Will attempt to load dataset at: {data_path}")

In [ ]:
import ssl4rs.data.parsers.fmow

parser = ssl4rs.data.parsers.fmow.DeepLakeParser(data_path)
parser.summary()

In [ ]:
# the default visualizer from deeplake doesn't work too well with sequences of images...
# (let's display some stuff manually instead)

import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import PIL.ImageDraw

max_display_count = 5

display_instance_idx = np.random.randint(len(parser))
print(f"Will display the different views of instance#{display_instance_idx}...")
instance_data = parser[display_instance_idx]
instance_view_count = instance_data["view_count"]
instance_views = instance_data["views"].data(aslist=True)["value"]  # list of rgb views (images)
instance_metadata = instance_data["metadata"]  # list of json metadata dicts of the instance
instance_bboxes = instance_data["bboxes"]  # list of LTRB, pixel-wise instance bounding boxes
instance_label = instance_data["label"]  # class label of the instance (e.g. "airport")
instance_subset = instance_data["subset"]  # subset label of the instance (e.g. "train")
instance_id = instance_data["instance"]  # unique name (str) for this particular instance

print(f"Instance: {instance_id} (class={instance_label}, subset={instance_subset})")
if instance_view_count > max_display_count:
    print(f"\tNumber of views: {instance_view_count} (will display first {max_display_count})")
else:
    print(f"\tNumber of views: {instance_view_count}")
display_count = min(max_display_count, instance_view_count)

fig, axes = plt.subplots(nrows=1, ncols=display_count, squeeze=False, figsize=(5 * display_count, 5))
for view_idx in range(display_count):
    view_img = PIL.Image.fromarray(instance_views[view_idx])
    drawer = PIL.ImageDraw.ImageDraw(view_img)
    bbox = instance_bboxes[view_idx]
    left, top = bbox[0], bbox[1]
    right, bottom = left + bbox[2], top + bbox[3]
    drawer.rectangle((left, top, right, bottom), width=10, outline="red")
    axes[0][view_idx].imshow(view_img, aspect="auto")